In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge as R
from sklearn.linear_model import Lasso as L
%matplotlib inline

In [9]:
#データセット読み込み
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
stadium = pd.read_csv("stadium.csv")
sample = pd.read_csv("sample_submit.csv", header=None)
cond = pd.read_csv("condition.csv")
cond_add = pd.read_csv("condition_add.csv")
data_2014 = pd.read_csv("2014_add.csv")
train_add = pd.read_csv("train_add.csv")

In [55]:
train.head()

,id,year,stage,match,gameday,time,home,away,stadium,tv
0,13994,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,スカパー／ｅ２／スカパー光／ＮＨＫ総合
1,13995,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ ４）／ＮＨＫ名古屋
2,13996,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,スカパー／ｅ２／スカパー光（Ｊ ＳＰＯＲＴＳ １）／ＮＨＫ大阪
3,13997,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,スカパー／ｅ２／スカパー光／ＮＨＫ広島
4,13998,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,スカパー／ｅ２／スカパー光（スカイ・Ａ ｓｐｏｒｔｓ＋）／ＮＨＫ札幌


In [54]:
y = train["y"]
train = train.drop("y", axis=1)
df = pd.concat([train, test])
df.isnull().sum()
df.head()

KeyError: 'y'

In [11]:
#merge df and cond by id
df = pd.merge(df, cond, left_on="id", right_on="id")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 0 to 2033
Data columns (total 40 columns):
id             2034 non-null int64
year           2034 non-null int64
stage          2034 non-null object
match          2034 non-null object
gameday        2034 non-null object
time           2034 non-null object
home           2034 non-null object
away           2034 non-null object
stadium        2034 non-null object
tv             2034 non-null object
home_score     2034 non-null int64
away_score     2034 non-null int64
weather        2034 non-null object
temperature    2034 non-null float64
humidity       2034 non-null object
referee        2034 non-null object
home_team      2034 non-null object
home_01        2034 non-null object
home_02        2034 non-null object
home_03        2034 non-null object
home_04        2034 non-null object
home_05        2034 non-null object
home_06        2034 non-null object
home_07        2034 non-null object
home_08        2034 non-null obj

In [14]:
df["time"] = df["time"].apply(lambda x: x.split(":")[0])
df["time"] = df["time"].astype(int)
df["humidity"] = df["humidity"].apply(lambda x: x.split("%")[0])
df["humidity"] = df["humidity"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 0 to 2033
Data columns (total 40 columns):
id             2034 non-null int64
year           2034 non-null int64
stage          2034 non-null object
match          2034 non-null object
gameday        2034 non-null object
time           2034 non-null int32
home           2034 non-null object
away           2034 non-null object
stadium        2034 non-null object
tv             2034 non-null object
home_score     2034 non-null int64
away_score     2034 non-null int64
weather        2034 non-null object
temperature    2034 non-null float64
humidity       2034 non-null int32
referee        2034 non-null object
home_team      2034 non-null object
home_01        2034 non-null object
home_02        2034 non-null object
home_03        2034 non-null object
home_04        2034 non-null object
home_05        2034 non-null object
home_06        2034 non-null object
home_07        2034 non-null object
home_08        2034 non-null objec

In [38]:
df["week"] = df["gameday"].apply(lambda x: list(x)[-2])
df = pd.merge(df, stadium, left_on="stadium", right_on="name")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2034 entries, 0 to 2033
Data columns (total 50 columns):
id             2034 non-null int64
year           2034 non-null int64
stage          2034 non-null object
match          2034 non-null object
gameday        2034 non-null object
time           2034 non-null int32
home           2034 non-null object
away           2034 non-null object
stadium        2034 non-null object
tv             2034 non-null object
home_score     2034 non-null int64
away_score     2034 non-null int64
weather        2034 non-null object
temperature    2034 non-null float64
humidity       2034 non-null int32
referee        2034 non-null object
home_team      2034 non-null object
home_01        2034 non-null object
home_02        2034 non-null object
home_03        2034 non-null object
home_04        2034 non-null object
home_05        2034 non-null object
home_06        2034 non-null object
home_07        2034 non-null object
home_08        2034 non-null objec

In [73]:
df_dummies = pd.get_dummies(df[["week", "stage", "temperature", "humidity", "time", "stadium", "weather", "home_team"]])

In [74]:
x_train = df_dummies.iloc[:1721, :]
x_test = df_dummies.iloc[1721:, :]
x_train.head()

,temperature,humidity,time,week_休,week_土,week_日,week_月,week_木,week_水,week_火,...,home_team_横浜ＦＣ,home_team_水戸ホーリーホック,home_team_浦和レッズ,home_team_清水エスパルス,home_team_湘南ベルマーレ,home_team_鹿島アントラーズ,home_team_ＦＣ岐阜,home_team_ＦＣ東京,home_team_ＦＣ町田ゼルビア,home_team_Ｖ・ファーレン長崎
0,3.8,66,14,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8.9,57,13,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.1,53,14,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10.4,55,19,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16.9,68,13,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 学習

In [68]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [75]:
#学習器の作成
def create_model():
    model = Sequential()
    model.add(Dense(150, input_dim=150))
    model.add(Activation("relu"))
    model.add(Dense(300))
    model.add(Activation("relu"))
    model.add(Dense(1))
    
    model.compile(loss="mean_squared_error", optimizer="adam")
    
    return model

In [76]:
trainX, testX, trainY, testY = train_test_split(x_train, y, random_state=709)

In [77]:
estimator = KerasRegressor(build_fn=create_model, epochs=100, batch_size=10, verbose=1)
estimator.fit(trainX, trainY)
predY = estimator.predict(testX)

mse = mean_squared_error(testY, predY)
print("keras REG mse = {:.4f}".format(mse ** 0.5))

Epoch 1/100
1290/1290 [==============================] - 1s 441us/step - loss: 139336330.0078
Epoch 2/100
1290/1290 [==============================] - 0s 220us/step - loss: 71932504.1783
Epoch 3/100
1290/1290 [==============================] - 0s 207us/step - loss: 71343124.6357
Epoch 4/100
1290/1290 [==============================] - 0s 205us/step - loss: 70238300.6667
Epoch 5/100
1290/1290 [==============================] - 0s 225us/step - loss: 69444022.3178
Epoch 6/100
1290/1290 [==============================] - 0s 233us/step - loss: 68469603.4690
Epoch 7/100
1290/1290 [==============================] - 0s 230us/step - loss: 67400556.5930
Epoch 8/100
1290/1290 [==============================] - 0s 232us/step - loss: 66214807.4574
Epoch 9/100
1290/1290 [==============================] - 0s 242us/step - loss: 65131369.71320s - loss: 592
Epoch 10/100
1290/1290 [==============================] - 0s 242us/step - loss: 63863022.5194
Epoch 11/100
1290/1290 [==============================

1290/1290 [==============================] - 0s 308us/step - loss: 38371808.7829
Epoch 88/100
1290/1290 [==============================] - 0s 257us/step - loss: 38217478.8857
Epoch 89/100
1290/1290 [==============================] - 0s 262us/step - loss: 38247276.1899
Epoch 90/100
1290/1290 [==============================] - 0s 260us/step - loss: 38461335.5271
Epoch 91/100
1290/1290 [==============================] - 0s 295us/step - loss: 38028461.2364
Epoch 92/100
1290/1290 [==============================] - 0s 289us/step - loss: 37887015.2791
Epoch 93/100
1290/1290 [==============================] - 0s 285us/step - loss: 38140237.2403
Epoch 94/100
1290/1290 [==============================] - 0s 265us/step - loss: 38166532.2500
Epoch 95/100
1290/1290 [==============================] - 0s 260us/step - loss: 38164945.9612
Epoch 96/100
1290/1290 [==============================] - 0s 257us/step - loss: 38016543.7752
Epoch 97/100
1290/1290 [==============================] - 0s 252us/step -